# Programação Matemática (SME0110)

## Trabalho de Otimização Inteira

Benício Januário, 12543843 \\
Henrique Bovo, 12542539 \\
Henrico Lazuroz, 12543502 \\
Victor Fernandes, 12675399 \\

## Tarefa 1: Escreva o modelo de localização de facilidades que minimiza os custos em linguagem de modelagem.

### Importação das instâncias

In [ ]:
!wget -q https://raw.githubusercontent.com/victorlfernandes/Trabalho-Programacao-Matematica/main/Adaptada-wlp01.txt
!wget -q https://raw.githubusercontent.com/victorlfernandes/Trabalho-Programacao-Matematica/main/Adaptada-wlp02.txt
!wget -q https://raw.githubusercontent.com/victorlfernandes/Trabalho-Programacao-Matematica/main/Adaptada-wlp03.txt
!wget -q https://raw.githubusercontent.com/victorlfernandes/Trabalho-Programacao-Matematica/main/Adaptada-wlp04.txt
!wget -q https://raw.githubusercontent.com/victorlfernandes/Trabalho-Programacao-Matematica/main/Adaptada-wlp05.txt

### Instalação do Solver SCIP

In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()

✨🍰✨ Everything looks OK!


In [ ]:
!conda install -y pyscipopt

Solving environment: \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / done


==> WARNING: A newer version of conda exists. <==
  current version: 23.1.0
  latest version: 23.10.0

Please update conda by running

    $ conda update -n base -c conda-forge conda

Or to minimize the number of packages updated during conda update use

     conda install conda=23.10.0



# All requested packages already installed.



In [ ]:
!mv /usr/local/lib/libscip.so.8.1 /usr/local/lib/libscip.so.8.0

mv: cannot stat '/usr/local/lib/libscip.so.8.1': No such file or directory


### Importação do Scip

In [ ]:
from pyscipopt import Model, quicksum, multidict
import time

### Criação do Modelo Inteiro no Scip

In [ ]:
def modeloLF_SCIP(numero_clientes, numero_facilidades, capacidade, custo_fixo, demanda, custo_cliente):

  model = Model("Localizacao_Facilidades_Scip")
  x, y = {}, {}
  clientes = range(numero_clientes)
  facilidades = range(numero_facilidades)

  for i in facilidades:

    # definindo as variáveis binárias que determinam se uma facilidade é aberta
    y[i] = model.addVar(vtype="BINARY", name="y(%s)"%i)

    for j in clientes:

      # definindo as variáveis contínuas que correspondem a fração da demanda do
      # cliente j atendida pela facilidade i
      x[i, j] = model.addVar(vtype="CONTINUOUS", name="x(%s,%s)"%(i, j))

  # definindo restrição 2, para que todos os clientes tenham sua demanda atendida
  for j in clientes:
    model.addCons(quicksum(x[i, j] for i in facilidades) == 1, "Demanda(%s)"%j)

  # definindo restrição 3, para que a capacidade das facilidades seja respeitada
  for i in facilidades:
    model.addCons(quicksum(demanda[j] * x[i, j] for j in clientes) <= (capacidade[i] * y[i]), "Capacidade(%s)"%i)

  # Definindo restrição 5, garantindo o dominio da variavel x
  for i in facilidades:
    for j in clientes:
      model.addCons(0 <= (x[i, j] <= 1), "X(%s)"%i)

  # definindo função objetivo
  model.setObjective(
      quicksum(custo_fixo[i] * y[i] for i in facilidades) +
      quicksum(custo_cliente[i, j] * x[i, j] for i in facilidades for j in clientes),
      "minimize"
  )

  model.data = x, y

  return model

## Tarefa 2: Explique brevemente por que essas restrições poderiam trazer melhorias. Para as instâncias disponibilizadas resolva o problema linearmente relaxado considerando o modelo (1) - (5). Em seguida, resolva novamente as instâncias trocando as restrições (3) pelas restrições (11).

In [ ]:
# Funcao para importar os dados das instancias

def ler_instancia(nome_arquivo):
    with open(nome_arquivo, 'r') as arquivo:
        linhas = arquivo.readlines()

    n, m = map(int, linhas[0].split())

    locais = linhas[1:n+1]
    clientes = linhas[n+1:]

    # Processando dados dos locais
    capacidades = [int(local.split()[0]) for local in locais]
    custos_fixos = [int(local.split()[1]) for local in locais]

    # Processando dados dos clientes
    demandas = [int(cliente.split()[0]) for cliente in clientes]

    # Processando custos de transporte
    custos_transporte = []
    for linha in clientes:
        custos_transporte.extend([int(custo) for custo in linha.split()[1:]])

    # Criando listas finais
    capi = capacidades
    fi = custos_fixos
    dj = demandas

    # Criando lista de custos de transporte
    cij = {}
    for i in range(m):
      for j in range(n):
        cij[(j, i)] = custos_transporte[j + (i*n)]

    return n, m, capi, fi, dj, cij

In [ ]:
# Modelo relaxado linermente (utilizando Scip)

def modeloLF_Relaxado_SCIP(numero_clientes, numero_facilidades, capacidade, custo_fixo, demanda, custo_cliente):

  model = Model("Localizacao_Facilidades_Relaxada_Scip")
  x, y = {}, {}
  clientes = range(numero_clientes)
  facilidades = range(numero_facilidades)

  for i in facilidades:

    # no modelo relaxado, yi passa a ser uma variavel continua entre 0 e 1, e nao binaria
    y[i] = model.addVar(vtype="CONTINUOUS", name="y(%s)"%i)

    for j in clientes:

      # definindo as variáveis contínuas que correspondem a fração da demanda do
      # cliente j atendida pela facilidade i
      x[i, j] = model.addVar(vtype="CONTINUOUS", name="x(%s,%s)"%(i, j))

  # definindo restrição 2, para que todos os clientes tenham sua demanda atendida
  for j in clientes:
    model.addCons(quicksum(x[i, j] for i in facilidades) == 1, "Demanda(%s)"%j)

  # definindo restrição 3, para que a capacidade das facilidades seja respeitada
  for i in facilidades:
    model.addCons(quicksum(demanda[j] * x[i, j] for j in clientes) <= (capacidade[i] * y[i]), "Capacidade(%s)"%i)

  # definindo restricao 4 (relaxao linear de yi)
  for i in facilidades:
    model.addCons(0 <= (y[i] <= 1), "Relaxacao(%s)"%i)

  # Definindo restrição 5, garantindo o dominio da variavel x
  for i in facilidades:
    for j in clientes:
      model.addCons(0 <= (x[i, j] <= 1), "X(%s)"%i)

  # definindo função objetivo
  model.setObjective(
      quicksum(custo_fixo[i] * y[i] for i in facilidades) +
      quicksum(custo_cliente[i, j] * x[i, j] for i in facilidades for j in clientes),
      "minimize"
  )

  model.data = x, y

  return model

In [ ]:
def print_result(model):
  print("Valor ótimo encontrado: ", model.getObjVal())
  print("Status: ", model.getStatus())
  print("Gap: ", model.getGap())
  print("Limitante Primal: ", model.getPrimalbound())
  print("Limitante Dual: ", model.getDualbound())

In [ ]:
# Resolucao do Modelo Relaxado para as Instancias
for i in range(5):
  print('Resolvendo instancia %s' % (i+1))
  n, m, capi, fi, dj, cij = ler_instancia('Adaptada-wlp0%s.txt' % (i+1))

  model = modeloLF_Relaxado_SCIP(m, n, capi, fi, dj, cij)
  model.setParam('limits/time', 300)
  start_time = time.time()
  model.optimize()
  elapsed_time = time.time() - start_time
  print("Tempo decorrido para Solução Ótima: {:.2f} segundos".format(elapsed_time))
  print_result(model)

Resolvendo instancia 1
Tempo decorrido para Solução Ótima: 19.98 segundos
Valor ótimo encontrado:  69005.35745604156
Status:  optimal
Gap:  0.0
Limitante Primal:  69005.35745604156
Limitante Dual:  69005.35745604156
Resolvendo instancia 2
Tempo decorrido para Solução Ótima: 36.61 segundos
Valor ótimo encontrado:  75840.63418600663
Status:  optimal
Gap:  0.0
Limitante Primal:  75840.63418600663
Limitante Dual:  75840.63418600663
Resolvendo instancia 3
Tempo decorrido para Solução Ótima: 81.13 segundos
Valor ótimo encontrado:  114633.6517158667
Status:  optimal
Gap:  0.0
Limitante Primal:  114633.6517158667
Limitante Dual:  114633.6517158667
Resolvendo instancia 4
Tempo decorrido para Solução Ótima: 270.21 segundos
Valor ótimo encontrado:  135069.02353311464
Status:  optimal
Gap:  0.0
Limitante Primal:  135069.02353311464
Limitante Dual:  135069.02353311464
Resolvendo instancia 5
Tempo decorrido para Solução Ótima: 309.65 segundos
Valor ótimo encontrado:  0.0
Status:  timelimit
Gap:  1e+

In [ ]:
# Modelo relaxado linearmente com novas restricoes

def modeloLF_Relaxado_Alternativo_SCIP(numero_clientes, numero_facilidades, capacidade, custo_fixo, demanda, custo_cliente):

  model = Model("Localizacao_Facilidades_Scip")
  x, y = {}, {}
  clientes = range(numero_clientes)
  facilidades = range(numero_facilidades)

  for i in facilidades:

    # no modelo relaxado, yi passa a ser uma variavel continua entre 0 e 1, e nao binaria
    y[i] = model.addVar(vtype="CONTINUOUS", name="y(%s)"%i)

    for j in clientes:

      # definindo as variáveis contínuas que correspondem a fração da demanda do
      # cliente j atendida pela facilidade i
      x[i, j] = model.addVar(vtype="CONTINUOUS", name="x(%s,%s)"%(i, j))

  # definindo restrição 2, para que todos os clientes tenham sua demanda atendida
  for j in clientes:
    model.addCons(quicksum(x[i, j] for i in facilidades) == 1, "Demanda(%s)"%j)

  # definindo restrição 3 alternativa, para que a capacidade das facilidades seja respeitada sem considerar o yi
  for idi, i in enumerate(facilidades):
    model.addCons(quicksum(demanda[idj] * x[i, j] for idj, j in enumerate(clientes)) <= capacidade[idi], "Capacidade(%s)"%i)

  # definindo restricao 4 (relaxao linear de yi)
  for i in facilidades:
    model.addCons(0 <= (y[i] <= 1), "Relaxacao(%s)"%i)

  # Definindo restrição 5, garantindo o dominio da variavel x
  for i in facilidades:
    for j in clientes:
      model.addCons(0 <= (x[i, j] <= 1), "X(%s)"%i)

  # definindo restricao 11, para melhorar o limitante dual do problema
  for j in clientes:
    for i in facilidades:
        model.addCons(x[i, j] <= y[i])

  # definindo função objetivo
  model.setObjective(
      quicksum(custo_fixo[id] * y[i] for id, i in enumerate(facilidades)) +
      quicksum(custo_cliente[i, j] * x[i, j] for i in facilidades for j in clientes),
      "minimize"
  )

  model.data = x, y

  return model

In [ ]:
# Resolucao do Modelo Relaxado Alternativo para as Instancias

for i in range(5):
  print('Resolvendo instancia %s' % (i+1))
  n, m, capi, fi, dj, cij = ler_instancia('Adaptada-wlp0%s.txt' % (i+1))

  model = modeloLF_Relaxado_Alternativo_SCIP(m, n, capi, fi, dj, cij)
  model.setParam('limits/time', 300)
  start_time = time.time()
  model.optimize()
  elapsed_time = time.time() - start_time
  print("Tempo decorrido para Solução Ótima: {:.2f} segundos".format(elapsed_time))
  print_result(model)

Resolvendo instancia 1
Tempo decorrido para Solução Ótima: 304.65 segundos
Valor ótimo encontrado:  0.0
Status:  timelimit
Gap:  1e+20
Limitante Primal:  1e+20
Limitante Dual:  0.0
Resolvendo instancia 2
Tempo decorrido para Solução Ótima: 301.62 segundos
Valor ótimo encontrado:  0.0
Status:  timelimit
Gap:  1e+20
Limitante Primal:  1e+20
Limitante Dual:  0.0
Resolvendo instancia 3
Tempo decorrido para Solução Ótima: 301.89 segundos
Valor ótimo encontrado:  0.0
Status:  timelimit
Gap:  1e+20
Limitante Primal:  1e+20
Limitante Dual:  0.0
Resolvendo instancia 4
Tempo decorrido para Solução Ótima: 300.47 segundos
Valor ótimo encontrado:  0.0
Status:  timelimit
Gap:  1e+20
Limitante Primal:  1e+20
Limitante Dual:  0.0
Resolvendo instancia 5


## Tarefa 3: Resolva as instâncias utilizando o Scip.

In [ ]:
for i in range(5):
  print('Resolvendo instancia %s' % (i+1))
  n, m, capi, fi, dj, cij = ler_instancia('Adaptada-wlp0%s.txt' % (i+1))

  model = modeloLF_SCIP(m, n, capi, fi, dj, cij)
  model.setParam('limits/time', 300)
  start_time = time.time()
  model.optimize()
  elapsed_time = time.time() - start_time
  print("Tempo decorrido para Solução Ótima: {:.2f} segundos".format(elapsed_time))
  print_result(model)

Resolvendo instancia 1
Tempo decorrido para Solução Ótima: 300.43 segundos
Valor ótimo encontrado:  69335.52771322845
Status:  timelimit
Gap:  0.004238953761785662
Limitante Primal:  69335.52771322845
Limitante Dual:  69042.85822961161
Resolvendo instancia 2
Tempo decorrido para Solução Ótima: 300.87 segundos
Valor ótimo encontrado:  75932.68186171414
Status:  timelimit
Gap:  0.0008253623817722075
Limitante Primal:  75932.68186171414
Limitante Dual:  75870.061566994
Resolvendo instancia 3
Tempo decorrido para Solução Ótima: 301.28 segundos
Valor ótimo encontrado:  114899.85410536746
Status:  timelimit
Gap:  0.001950377859463004
Limitante Primal:  114899.85410536746
Limitante Dual:  114676.19219909482
Resolvendo instancia 4
Tempo decorrido para Solução Ótima: 302.45 segundos
Valor ótimo encontrado:  140164.59716159131
Status:  timelimit
Gap:  0.03747843227871169
Limitante Primal:  140164.59716159131
Limitante Dual:  135101.21540910937
Resolvendo instancia 5
Tempo decorrido para Solução 

## Tarefa 4: Resolva as instâncias utilizando o Gurobi.

### Instalando o Gurobi

In [ ]:
%pip install -q gurobipy

### Importação do Gurobi

In [ ]:
import gurobipy as gp
from gurobipy import *

# Configurando licenca de estudante
options = {
    "WLSACCESSID": "a3bc1655-41fe-4283-85b1-1e49702c7e03",
    "WLSSECRET": "966b264f-b9fc-4f6e-8c84-e710200d4142",
    "LICENSEID": 2445446,
}

### Criação do Modelo Inteiro no Gurobi

In [ ]:
def modeloLF_Gurobi(numero_clientes, numero_facilidades, capacidade, custo_fixo, demanda, custo_cliente):
    with gp.Env(params=options) as env, gp.Model(name="Localizacao_Facilidades_Gurobi", env=env) as model:
      x, y = {}, {}
      clientes = range(numero_clientes)
      facilidades = range(numero_facilidades)

      for i in facilidades:
          # definindo as variáveis binárias que determinam se uma facilidade é aberta
          y[i] = model.addVar(vtype=GRB.BINARY, name="y(%s)" % i)

          for j in clientes:
              # definindo as variáveis contínuas que correspondem à fração da demanda do
              # cliente j atendida pela facilidade i
              x[i, j] = model.addVar(vtype=GRB.CONTINUOUS, name="x(%s,%s)" % (i, j))

      # definindo restrição 2, para que todos os clientes tenham sua demanda atendida
      for j in clientes:
          model.addConstr(quicksum(x[i, j] for i in facilidades) == 1, "Demanda(%s)" % j)

      # definindo restrição 3, para que a capacidade das facilidades seja respeitada
      for i in facilidades:
          model.addConstr(quicksum(demanda[j] * x[i, j] for j in clientes) <= (capacidade[i] * y[i]), "Capacidade(%s)" % i)

      # Definindo restrição 5, garantindo o dominio da variavel x
      for i in facilidades:
        for j in clientes:
          model.addConstr(x[i, j] >= 0, "LowerBound_X(%s,%s)" % (i, j))
          model.addConstr(x[i, j] <= 1, "UpperBound_X(%s,%s)" % (i, j))

      # definindo função objetivo
      model.setObjective(
          quicksum(custo_fixo[i] * y[i] for i in facilidades) +
          quicksum(custo_cliente[i, j] * x[i, j] for i in facilidades for j in clientes),
          GRB.MINIMIZE
      )

      model.update()

      model.setParam('TimeLimit', 300)

      start_time = time.time()
      model.optimize()
      elapsed_time = time.time() - start_time
      print("Tempo decorrido para Solução Ótima: {:.2f} segundos".format(elapsed_time))

      print("Valor ótimo encontrado: ", model.objVal)
      print("Status: ", model.status)
      print("Gap: ", model.MIPGap)
      print("Limitante Primal: ", model.objBound)
      print("Limitante Dual: ", model.ObjBoundC)

      return model

In [ ]:
# Resolucao do Modelo Relaxado Alternativo para as Instancias

for i in range(5):
    print('Resolvendo instancia %s' % (i+1))
    n, m, capi, fi, dj, cij = ler_instancia('Adaptada-wlp0%s.txt' % (i+1))

    modeloLF_Gurobi(m, n, capi, fi, dj, cij)

Resolvendo instancia 1
Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2445446
Academic license 2445446 - for non-commercial use only - registered to he___@usp.br
Set parameter TimeLimit to value 300
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Academic license 2445446 - for non-commercial use only - registered to he___@usp.br
Optimize a model with 301351 rows, 150500 columns and 601250 nonzeros
Model fingerprint: 0x2247b9ee
Variable types: 150250 continuous, 250 integer (250 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [1e+00, 1e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 186346.00000
Presolve removed 300500 rows and 0 columns
Presolve time: 0.30s
Presolved: 85

## Tarefa 5: Aplicação do Problema de Localização de Facilidades.

### Aplicação: Localização de Hospitais com base na demanda da População de São Carlos
Uma aplicação do problema de localização de facilidades seria a escolha dos melhores bairros em São Carlos para se abrir novos hospitais públicos, respeitando a demanda da população local.

## Tarefa 6: Toy Problem

Vamos considerar que São Carlos possui dois hospitais, Santa Casa (0) e o Hospital da Federal (1), e queremos ajudar a prefeitura a abrir um novo hospital (2) na cidade.

Por se tratar de um Toy Problem, vamos considerar apenas 10 pacientes e 5 bairros.

In [ ]:
# Dados do Toy Problem
import random

def gera_massa_de_dados(n, m):
    capacidade = [random.randint(5, 150) for _ in range(n)]
    custo_fixo = [random.randint(10, 50) for _ in range(n)]

    matriz_custos = [[random.randint(1, 10) for _ in range(n)] for _ in range(m)]
    demandas = [random.randint(1, 5) for _ in range(m)]

    return capacidade, custo_fixo, matriz_custos, demandas

# Gerar nova massa de dados
n = 5
m = 10
capacidade, custo_fixo, matriz_custos, demandas = gera_massa_de_dados(n, m)

# Exibir os dados gerados
print("Capacidade:", capacidade)
print("Custo Fixo:", custo_fixo)
print("Matriz de Custos:")
for row in matriz_custos:
    print(row)
print("Demandas:", demandas)


def escreve_instancia(n, m, f, c, d, cap):
  with open('InstanciaToyProblem.txt', 'w') as arquivo:
    arquivo.write('%s %s\n' % (n, m))
    for i in range(n):
      arquivo.write('%s %s\n' % (cap[i], f[i]))
    for j in range(m):
        arquivo.write('%s' % d[j])
        for i in range(n):
          arquivo.write(' %s' % c[j][i])
        arquivo.write('\n')

escreve_instancia(n, m, custo_fixo, matriz_custos, demandas, capacidade)

Capacidade: [52, 149, 14, 146, 8]
Custo Fixo: [15, 11, 34, 31, 19]
Matriz de Custos:
[3, 2, 8, 4, 5]
[7, 8, 5, 6, 1]
[4, 3, 7, 10, 6]
[6, 2, 5, 7, 1]
[5, 6, 5, 3, 8]
[2, 4, 2, 8, 7]
[4, 4, 7, 4, 7]
[9, 10, 10, 9, 4]
[2, 2, 7, 5, 7]
[4, 8, 7, 7, 9]
Demandas: [5, 5, 3, 1, 5, 2, 2, 1, 4, 3]


In [ ]:
# Criação do Modelo utilizando o SCIP

from pyscipopt import Model, quicksum, multidict


def modelo_hospitais_sao_carlos(n, m, fi, cij, dj, Capi):
  model = Model("Localizacao_Hospital_SC_Scip")
  x, y = {}, {}
  bairros = range(1, n + 1)
  pacientes = range(1, m + 1)

  for i in bairros:
      # Definindo as variáveis binárias que determinam se um hospital é aberto
      y[i] = model.addVar(vtype="BINARY", name="y(%s)" % i)

      for j in pacientes:
          # Definindo as variáveis contínuas que correspondem à fração da demanda do
          # paciente j atendida pelo hospital i
          x[i, j] = model.addVar(vtype="CONTINUOUS", name="x(%s,%s)" % (i, j))

  # Definindo restrição 2, para que todos os pacientes tenham sua demanda por atendimento médico respeitada
  for j in pacientes:
      model.addCons(quicksum(x[i, j] for i in bairros) == 1, "Demanda(%s)" % j)

  # Definindo restrição 3, para que a capacidade dos hospitais seja respeitada
  for i in bairros:
    model.addCons(quicksum(dj[j-1] * x[i, j] for j in pacientes) <= Capi[i-1] * y[i], "Capacidade(%s)" % i)

  # Definindo restrição 4, para sempre abrir os hospitais 1 e 2 e um novo
  model.addCons(y[1] == 1, "SantaCasa")
  model.addCons(y[2] == 1, "Federal")
  model.addCons(quicksum(y[i] for i in bairros) == 3, "Novo")

  # Definindo restrição 5, garantindo o dominio da variavel x
  for i in bairros:
    for j in pacientes:
      model.addCons(0 <= (x[i, j] <= 1), "X(%s)"%i)

  # Definindo função objetivo
  model.setObjective(
      quicksum(fi[i-1] * y[i] for i in bairros) +
      quicksum(cij[i-1, j-1] * x[i, j] for i in bairros for j in pacientes),
      "minimize"
  )

  model.data = x, y

  return model

In [ ]:
# Resolvendo o Toy Problem
n, m, Capi, fi, dj, cij = ler_instancia('InstanciaToyProblem.txt')

model = modelo_hospitais_sao_carlos(n, m, fi, cij, dj, Capi)

start_time = time.time()
model.optimize()
elapsed_time = time.time() - start_time
print("Tempo decorrido para Solução Ótima: {:.2f} segundos".format(elapsed_time))

Tempo decorrido para Solução Ótima: 0.01 segundos


In [ ]:
EPS = 1.e-6
x, y = model.data
print("Status: ", model.getStatus())
print("Gap: ", model.getGap())
print("Limitante Primal: ", model.getPrimalbound())
print("Limitante Dual: ", model.getDualbound())
hospital_aberto = [j for j in y if model.getVal(y[j]) > EPS]
atendimentos = [(i,j) for (i,j) in x if model.getVal(x[i,j]) > EPS]
print("Valor ótimo encontrado: ", model.getObjVal())
print("Hospital Aberto: ", hospital_aberto[2:])
hospital_map = {1: 'Hospital Santa Casa', 2: 'Hospital da Federal'}

atendimentos_map = {'Hospital Santa Casa': [], 'Hospital da Federal': [], 'Novo Hospital': []}
for atendimento in atendimentos:
    hospital, paciente = atendimento
    nome_hospital = hospital_map.get(hospital, f'Novo Hospital')

    atendimentos_map[nome_hospital].append(f'Paciente {paciente}')

for hospital, atendimentos in atendimentos_map.items():
    pacientes = ', '.join(map(str, atendimentos))
    print(f"{hospital}: {pacientes}")

Status:  optimal
Gap:  0.0
Limitante Primal:  73.0
Limitante Dual:  73.0
Valor ótimo encontrado:  73.0
Hospital Aberto:  [5]
Hospital Santa Casa: Paciente 5, Paciente 6, Paciente 7, Paciente 9, Paciente 10
Hospital da Federal: Paciente 1, Paciente 3
Novo Hospital: Paciente 2, Paciente 4, Paciente 8
